# BI output results and speed comparaison
## 1. Continuous variable (model 4.3)
### Output comparaison

In [19]:
import time
from src.main import*
## Model m4.3
d = pd.read_csv('./data/Howell1.csv', sep=';')
d = d[d.age > 18]
#self.df["weight.per.g"].pipe(lambda x: (x - x.mean()) / x.std())
d.weight = d.weight - d.weight.mean()
d.age = d.age - d.age.mean()
formula = dict(main1 = 'height ~ Normal(mu,sigma)',
            likelihood = 'mu ~ alpha + beta * weight',
            prior1 = 'sigma ~ Uniform(0,50)',
            prior2 = 'alpha ~ Normal(178,20)',
            prior3 = 'beta ~ Normal(0,1)')    

self = model(formula, df = d, float = 32)
self.fit(observed_data = dict(height =d.height.astype('float32').values),
                                           num_results = 2000, num_burnin_steps=500, num_adaptation_steps=400, num_chains=4)       
self.summary(round_to = 'none')      

2024-03-11 13:55:19.174226: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-11 13:55:19.174293: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-11 13:55:19.174306: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-11 13:55:19.174467: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-11 13:55:19.174479: I tensorflow/core/common_runtime/gpu/gpu

,mean,sd,hdi_5.5%,hdi_94.5%
sigma[0],5.143253,0.194696,4.815463,5.434054
beta[0],0.905713,0.042139,0.838150,0.973281
alpha[0],154.654694,0.279546,154.202972,155.086349


In [8]:
import stan
import nest_asyncio
nest_asyncio.apply()
stan_code = """
data{
  vector[346] height;
  vector[346] weight;
}
parameters{
  real a;
  real<lower=0> b;
  real<lower=0,upper=50> sigma;
}
model{
  vector[346] mu;
  sigma ~ uniform( 0 , 50 );
  b ~ lognormal( 0 , 1 );
  a ~ normal( 178 , 20 );
  for ( i in 1:346 ) {
    mu[i] = a + b* weight[i] ;
  }
  height ~ normal( mu , sigma );
}
"""
data = {
  'height': d.height.values,
  'weight': d.weight.values,
}
stan_model = stan.build(stan_code, data = data)
fit = stan_model.sample(num_chains=4, num_samples=2000, num_warmup = 500)
df = fit.to_frame()

AttributeError: 'DataFrame' object has no attribute 'height'

In [26]:
pd.DataFrame(
    {
        "tfp": self.summary(round_to = 'none')['mean'],
        "pystan": [df.sigma.mean(),  df.b.mean(), df.a.mean()]
    })

,tfp,pystan
sigma[0],5.143253,5.143658
beta[0],0.905713,0.904088
alpha[0],154.654694,154.648417


## 2. Categorical variable (model 5.9)
### Output comparaisons

In [1]:
from src.main import*
self = model()
self.import_csv('./data/milk.csv', sep = ';')
self.df["K"] = self.df["kcal.per.g"].pipe(lambda x: (x - x.mean()) / x.std())
self.index(cols = "clade")
formula = dict(main = 'K ~ Normal(mu,sigma)',
            likelihood = 'mu ~ alpha[index_clade]',
            prior1 = 'alpha~ Normal(0,0.5)',
            prior2 = 'sigma ~ Exponential(1)') 

self.f = formula
self.build_model()
self.fit(observed_data = dict(K =self.df.K.astype('float32').values),
                                           num_results = 2000, num_burnin_steps=500, num_adaptation_steps=400, num_chains=4)

2024-03-11 13:59:54.945989: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-11 13:59:54.966675: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-11 13:59:54.966697: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-11 13:59:54.967219: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-11 13:59:54.970806: I tensorflow/core/platform/cpu_feature_guar

,mean,sd,hdi_5.5%,hdi_94.5%
sigma[0],0.795420,0.113958,0.612368,0.957918
alpha[0],-0.463743,0.237253,-0.814464,-0.050572
alpha[1],0.350829,0.229937,-0.043127,0.690739
alpha[2],0.642362,0.290458,0.163980,1.099852
alpha[3],-0.547165,0.290076,-0.992915,-0.058458


In [ ]:
import stan
import nest_asyncio
nest_asyncio.apply()
stan_code = """
data{
    vector[29] K;
    array[29] int clade_id;
}
parameters{
    vector[4] a;
    real<lower=0> sigma;
}
model{
    vector[29] mu;
    sigma ~ exponential( 1 );
    a ~ normal( 0 , 0.5 );
    for ( i in 1:29 ) {
        mu[i] = a[clade_id[i]];
    }
    K ~ normal( mu , sigma );
}
"""
data = {
  'clade_id': self.df.index_clade.values+1,
  'K': self.df.K.values,
}
stan_model = stan.build(stan_code, data = data)
fit = stan_model.sample(num_chains=4, num_samples=2000, num_warmup = 500)
df = fit.to_frame()

In [4]:
pd.DataFrame(
    {
        "tfp": self.summary(round_to = 'none')['mean'],
        "pystan": [df.sigma.mean(),  df['a.1'].mean(), df['a.2'].mean(), df['a.3'].mean(), df['a.4'].mean()]
    })

,tfp,pystan
sigma[0],0.795420,0.801740
alpha[0],-0.463743,-0.462021
alpha[1],0.350829,0.346709
alpha[2],0.642362,0.637985
alpha[3],-0.547165,-0.545071


## 3. Continuous interactions terms (model 8.3)
### Output comparaisons

In [6]:
from src.main import*
d = pd.read_csv('./data/tulips.csv', sep = ';')
d["blooms_std"] = d.blooms / d.blooms.max()
d["water_cent"] = d.water - d.water.mean()
d["shade_cent"] = d.shade - d.shade.mean()

formula = dict(
            main = 'blooms_std ~ Normal( mu , sigma ) ',
            likelihood ='mu ~ a + bw*water_cent + bs*shade_cent + bws*water_cent*shade_cent' ,
            prior1 = 'a ~ Normal( 0.5 , 0.25 ) ',
            prior2 = 'bw ~ Normal( 0 , 0.25 ) ',
            prior3 = 'bs ~ Normal( 0 , 0.25 ) ',
            prior4 = 'bws ~ Normal( 0 , 0.25 ) ',
            prior5 = 'sigma ~ Exponential( 1 )',
            )
m8_5 = model(formula, d)
m8_5.fit(observed_data = dict(blooms_std =d.blooms_std.astype('float32').values),
                                           num_results = 2000, num_burnin_steps=500, num_adaptation_steps=400, num_chains=4)

2024-03-11 15:30:32.065036: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-11 15:30:32.065104: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-11 15:30:32.065118: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-11 15:30:32.065379: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-11 15:30:32.065394: I tensorflow/core/common_runtime/gpu/gpu

In [4]:
import stan
import nest_asyncio
nest_asyncio.apply()

stan_code = """
data{
    vector[27] blooms_std;
    array[27] int shade_cent;
    array[27] int water_cent;
}
parameters{
    real a;
    real bw;
    real bs;
    real bws;
    real<lower=0> sigma;
}
model{
    vector[27] mu;
    sigma ~ exponential( 1 );
    bws ~ normal( 0 , 0.25 );
    bs ~ normal( 0 , 0.25 );
    bw ~ normal( 0 , 0.25 );
    a ~ normal( 0.5 , 0.25 );
    for ( i in 1:27 ) {
        mu[i] = a + bw * water_cent[i] + bs * shade_cent[i] + bws * water_cent[i] * shade_cent[i];
    }
    blooms_std ~ normal( mu , sigma );
}
"""

data = {
    'blooms_std' : d["blooms_std"].values,
    "water_cent": d["water_cent"].values.astype(int),
    "shade_cent": d["shade_cent"].values.astype(int),
}

stan_model = stan.build(stan_code, data = data)
fit = stan_model.sample(num_chains=4, num_samples=2000, num_warmup = 500)
df = fit.to_frame()

Building...



Building: found in cache, done.Sampling:   0%Building: found in cache, done.Sampling:   0%
Sampling:  25% (2500/10000)
Sampling:  50% (5000/10000)
Sampling:  75% (7500/10000)
Sampling: 100% (10000/10000)
Sampling: 100% (10000/10000), done.
Messages received during sampling:
  Gradient evaluation took 0.000127 seconds
  1000 transitions using 10 leapfrog steps per transition would take 1.27 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 2.6e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.26 seconds.
  Adjust your expectations accordingly!
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: normal_lpdf: Scale parameter is 0, but must be positive! (in '/tmp/httpstan_vzxf68w9/model_u4xrvgfe.stan', line 24, column 4 to column 38)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sample

In [26]:
pd.DataFrame(
    {
        "tfp": m8_5.summary(round_to='none')['mean'],
        "pystan": [df.sigma.mean(),  df['bws'].mean(), df['bw'].mean(), df['bs'].mean(), df['a'].mean()]
    })

,tfp,pystan
sigma[0],0.144031,0.142922
bws[0],-0.141373,-0.142347
bw[0],0.205713,0.205934
bs[0],-0.112869,-0.112779
a[0],0.358613,0.358637


## 4. Binomial model
### Output comparaisons

In [1]:
from src.main import*
d = pd.read_csv('./data/chimpanzees.csv', sep = ';')
d["treatment"] = 1 + d.prosoc_left + 2 * d.condition
d["side"] = d.prosoc_left  # right 0, left 1
d["cond"] = d.condition  # no partner 0, partner 1

d_aggregated = (
    d.groupby(["treatment", "actor", "side", "cond"])["pulled_left"].sum().reset_index()
)
d_aggregated.rename(columns={"pulled_left": "left_pulls"}, inplace=True)
d_aggregated["actor_id"] = d_aggregated["actor"].values - 1

formula = dict(
    main = 'pulled_left ~ Binomial( 1 , logits = p )' ,
    likelihood = 'p ~ a' ,
    prior1 = 'a ~ Normal( 0 , 10 )'
)

m11_1 = model(formula, d)
m11_1.fit(observed_data = dict(pulled_left =d.pulled_left.astype('float32').values),
                                           num_results = 2000, num_burnin_steps=500, num_adaptation_steps=400, num_chains=4)

2024-03-11 15:42:11.690139: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-11 15:42:11.710628: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-11 15:42:11.710647: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-11 15:42:11.711154: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-11 15:42:11.714917: I tensorflow/core/platform/cpu_feature_guar

In [4]:
import stan
import nest_asyncio
nest_asyncio.apply()

stan_code = """
data{
    array[504] int pulled_left;
}
parameters{
    real a;
}
model{
    real p;
    a ~ normal( 0 , 10 );
    p = a;
    p = inv_logit(p);
    pulled_left ~ binomial( 1 , p );
}
"""

data = {
    'pulled_left' : d["pulled_left"].values.astype(int)
}

stan_model = stan.build(stan_code, data = data)
fit = stan_model.sample(num_chains=4, num_samples=2000, num_warmup = 500)
df = fit.to_frame()

Building...



Building: found in cache, done.Building: found in cache, done.Sampling:   0%
Sampling:  25% (2500/10000)
Sampling:  50% (5000/10000)
Sampling:  75% (7500/10000)
Sampling: 100% (10000/10000)
Sampling: 100% (10000/10000), done.
Messages received during sampling:
  Gradient evaluation took 4e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.04 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 5e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.05 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 4e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.04 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 5e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.05 seconds.
  Adjust your expectations accordingly!


In [7]:
pd.DataFrame(
    {
        "tfp": m11_1.summary(round_to='none')['mean'],
        "pystan": [df.a.mean()]
    })

,tfp,pystan
a[0],0.320727,0.318133


##  5. Binomial with indices
### Output comparaisons

In [31]:
from src.main import*
d = pd.read_csv('./data/chimpanzees.csv', sep = ';')
d.actor = d.actor - 1
d["treatment"] = d.prosoc_left + 2 * d.condition
d[["actor", "prosoc_left", "condition", "treatment"]]

formula = dict(
    main = 'pulled_left ~ Binomial(1 , p )' ,
    likelihood = 'p ~ a[actor] + b[treatment]' ,
    prior1 = 'a ~ Normal(0, 1.5)',
    prior2 = 'b ~ Normal(0, 0.5)'
)

m11_4 = model(formula, d, float = 32)
m11_4.fit(observed_data = dict(pulled_left =d.pulled_left.astype('float32').values),
                                           num_results = 2000, num_burnin_steps=500, num_adaptation_steps=400, num_chains=4)


2024-03-11 16:03:17.242708: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-11 16:03:17.242776: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-11 16:03:17.242789: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-11 16:03:17.242915: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-11 16:03:17.242921: I tensorflow/core/common_runtime/gpu/gpu

In [32]:
import stan
import nest_asyncio
nest_asyncio.apply()

stan_code = """
data{
    array[504] int pulled_left;
    array[504] int treatment;
    array[504] int actor;
}
parameters{
    vector[7] a;
    vector[4] b;
}
model{
    vector[504] p;
    b ~ normal( 0 , 0.5 );
    a ~ normal( 0 , 1.5 );
    for ( i in 1:504 ) {
        p[i] = a[actor[i]] + b[treatment[i]];
        p[i] = inv_logit(p[i]);
    }
    pulled_left ~ binomial( 1 , p );
}
generated quantities{
    vector[504] log_lik;
    vector[504] p;
    for ( i in 1:504 ) {
        p[i] = a[actor[i]] + b[treatment[i]];
        p[i] = inv_logit(p[i]);
    }
    for ( i in 1:504 ) log_lik[i] = binomial_lpmf( pulled_left[i] | 1 , p[i] );
}
"""

data = {
    'pulled_left' : d["pulled_left"].values.astype(int),
    'treatment' : d["treatment"].values.astype(int) +1,
    'actor' : d["actor"].values.astype(int) +1
}

stan_model = stan.build(stan_code, data = data)
fit = stan_model.sample(num_chains=4, num_samples=2000, num_warmup = 500)
df = fit.to_frame()

Building...



Building: found in cache, done.Building: found in cache, done.Sampling:   0%
Sampling:  25% (2500/10000)
Sampling:  50% (5000/10000)
Sampling:  75% (7500/10000)
Sampling: 100% (10000/10000)
Sampling: 100% (10000/10000), done.
Messages received during sampling:
  Gradient evaluation took 2.4e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.24 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 2.1e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.21 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 1.9e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.19 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 2.4e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.24 seconds.
  Adjust your expectations accordingly!


In [40]:
pd.DataFrame(
    {
        "tfp": m11_4.summary(round_to='none')['mean'],
        "pystan": [df['b.1'].mean(), df['b.2'].mean(), df['b.3'].mean(), df['b.4'].mean(),
                   df['a.1'].mean(), df['a.2'].mean(), df['a.3'].mean(), df['a.4'].mean(), df['a.5'].mean(), df['a.6'].mean(), df['a.7'].mean()]
    })

,tfp,pystan
b[0],-0.039967,-0.028233
b[1],0.480540,0.491561
b[2],-0.379118,-0.372934
b[3],0.369211,0.379449
a[0],-0.453946,-0.456491
a[1],3.900515,3.904893
a[2],-0.748389,-0.759698
a[3],-0.749001,-0.759213
a[4],-0.440749,-0.459564
a[5],0.478016,0.470364


## 6. Poisson
### Output comparaison

In [41]:
from src.main import*
d = pd.read_csv('./data/Kline.csv', sep = ';')
d["P"] = d.population.pipe(np.log).pipe(lambda x: (x - x.mean()) / x.std())
d["cid"] = (d.contact == "high").astype(int)
d['pLog'] = tf.math.log(d.P).numpy()
formula = dict(main = 'total_tools ~ Poisson(log_rate = LL)',
               likelihood = 'LL ~ alpha',
               prior1 = 'alpha ~ Normal(3,0.5)')
m11_9 = model(formula, d)
m11_9.fit(observed_data = dict(total_tools =d.total_tools.astype('float32').values),
                                           num_results = 2000, num_burnin_steps=500, num_adaptation_steps=400, num_chains=4)


2024-03-11 16:09:57.742119: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-11 16:09:57.742189: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-11 16:09:57.742206: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-11 16:09:57.742398: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-11 16:09:57.742410: I tensorflow/core/common_runtime/gpu/gpu

# Todo, param in model 2 to get in model 1

In [70]:
formula = dict(main = 'y~Normal(m,s)',
            likelihood = 'm ~  alpha + beta*z',
            prior1 = 's~Exponential(1)',
            prior2 = 'alpha ~ Normal(0,1)',
            prior3 = 'beta ~ Normal(0,1)',
            
            main1 = 'z~Normal(m2,s2)',
            likelihood2 = 'm2 ~ alpha2 + beta2',
            prior4 = 's2~Exponential(1)',
            prior5 = 'alpha2 ~ Normal(0,1)',
            prior6 = 'beta2 ~ Normal(0,1)') 

self = model()
self.f = formula
self.get_model_info()
self.merge()
self.write_priors()
self.write_mains()
self.build_tensor()
self.main_dict

2024-03-11 16:20:35.669067: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-11 16:20:35.669120: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-11 16:20:35.669137: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-11 16:20:35.669247: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-11 16:20:35.669252: I tensorflow/core/common_runtime/gpu/gpu

{'y': "lambda s, alpha, beta : tfd.Independent(tfd.Normal(alpha+beta*z,s, name ='main'), reinterpreted_batch_ndims=1)",
 'z': "lambda s2, alpha2, beta2 : tfd.Independent(tfd.Normal(alpha2+beta2,s2, name ='main1'), reinterpreted_batch_ndims=1)"}

In [86]:
self.mains[key]['likelihood(s)']

{'likelihood': {'input': 'm ~  alpha + beta*z',
  'output': 'm',
  'formula': 'alpha+beta*z',
  'args': array(['alpha', 'beta', 'z'], dtype='<U5'),
  'likelihood(s)': {},
  'params': {'args': ['alpha', 'beta', 'z'], 'kwargs': {}},
  'distribution(s)': [],
  'prior(s)': {'prior2': {'input': 'alpha ~ Normal(0,1)',
    'output': 'alpha',
    'formula': 'tfd.Normal(0,1)',
    'args': array(['0', '1', 'Normal'], dtype='<U6'),
    'likelihood(s)': {},
    'params': {'args': ['0', '1', 'Normal'], 'kwargs': {}},
    'distribution(s)': ['tfd.Normal'],
    'prior(s)': {}},
   'prior3': {'input': 'beta ~ Normal(0,1)',
    'output': 'beta',
    'formula': 'tfd.Normal(0,1)',
    'args': array(['0', '1', 'Normal'], dtype='<U6'),
    'likelihood(s)': {},
    'params': {'args': ['0', '1', 'Normal'], 'kwargs': {}},
    'distribution(s)': ['tfd.Normal'],
    'prior(s)': {}}}}}

In [84]:
key = 'main'
len(self.mains[key]) 
text = "lambda "
lambda_params = []
len(self.mains[key]['prior(s)']) 
for k in self.mains[key]['prior(s)'].keys():
    text = text + self.mains[key]['prior(s)'][k]["output"] + ', '
    lambda_params.append(self.mains[key]['prior(s)'][k]["output"])

len(self.mains[key]['likelihood(s)'])
for a in self.mains[key]['likelihood(s)'].keys():
                       # Check its priors and add them to lambda
                       if len(self.mains[key]['likelihood(s)'][a]['prior(s)']) > 0:
                           for b in  self.mains[key]['likelihood(s)'][a]['prior(s)'].keys():
                               if  self.mains[key]['likelihood(s)'][a]['prior(s)'][b]["output"] not in lambda_params:
                                   text = text + self.mains[key]['likelihood(s)'][a]['prior(s)'][b]["output"]  + ', '
text

'lambda s, alpha, beta, '